# NYPD Criminal Court Summons (Historic):

a. To get the required columns, use this module:


1.   get_area_of_interest(df_spark, interested_columns)


b. Preprocessing pipeline: Pass your data through these functions. (if your columns fall in those categories)

1.   valid_date_check(date)
2.   valid_time_check(time)
3.   reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index)
4.   refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None)
5.   refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None)
6.   refine_precinct_jur(df_spark, precinct=None, Jur_code=None)



In [2]:
!pip install pyspark
!pip install openclean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.8/260.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 21.7 MB/s eta 0:00:00
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=155030 sha256=05ba2b5128e725c803a52c7c147005968f281cf7e24ca826b9c13e5358

In [4]:
#importing packages required
from pyspark import SparkContext, SparkConf
import os
import requests
from six.moves import urllib
import sys
import pandas as pd
import matplotlib
import matplotlib as plt
import numpy as np
import scipy as sp
import IPython
from IPython import display
import sklearn
import random
import time
import warnings
import re
import matplotlib.pyplot as plt
%matplotlib inline
from openclean.pipeline import stream
from openclean.profiling.column import DefaultColumnProfiler
from openclean.data.source.socrata import Socrata
from openclean.pipeline import stream
from openclean.function.eval.datatype import IsDatetime
import datetime
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType

In [5]:
from geopy.geocoders import ArcGIS
geocoder=ArcGIS()
#example:
geocoder.reverse('40.61157006600007, -73.74736517199995')

Location(808 Redfern Ave, Far Rockaway, New York 11691, USA, (40.611614718328, -73.747382377557, 0.0))

In [10]:
# API URL
fn_src = 'https://data.cityofnewyork.us/resource/sv2w-rv3k.csv?$limit=10000'
fn_dst = '/content/NYPD_CSummons_Historic_10k.csv'

# Download the file if not already present
if not os.path.isfile(fn_dst):
    print('Fetching file. This may take a while...', fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File has been downloaded:', fn_dst)
else:
    print('File already exists:', fn_dst)

# Initialize Spark
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Read CSV into Spark DataFrame
df_spark = spark.read.option("header", True).csv(fn_dst, inferSchema=True)

# Summary
print("Number of rows:", df_spark.count())
df_spark.printSchema()
df_spark.show(5)

Fetching file. This may take a while... /content/NYPD_CSummons_Historic_10k.csv
File has been downloaded: /content/NYPD_CSummons_Historic_10k.csv
Number of rows: 10000
root
 |-- summons_key: integer (nullable = true)
 |-- summons_date: timestamp (nullable = true)
 |-- offense_description: string (nullable = true)
 |-- law_section_number: string (nullable = true)
 |-- law_description: string (nullable = true)
 |-- summons_category_type: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- race: string (nullable = true)
 |-- jurisdiction_code: integer (nullable = true)
 |-- boro: string (nullable = true)
 |-- precinct_of_occur: integer (nullable = true)
 |-- x_coordinate_cd: integer (nullable = true)
 |-- y_coordinate_cd: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- geocoded_column: string (nullable = true)

+-----------+-------------------+--------------------+--------

In [ ]:
#similarly, lets get them into pyspark rdd
def get_area_of_interest(df_spark, interested_columns):
  df_spark=df_spark.select(interested_columns)
  return df_spark

# 2. Module for date related columns

As the dataset is for the data from 2006 to 2025, we can see that there is data from unknown format of "1010-05-14" to the year 2025. We need to clean this. Over here, we remove the null values where the complaint date is <2006.

In [1]:
# fileName='1010-05-14 00:00:00'
# # matches=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", fileName)
# re.search(r'([0-9]{4}\-[0-9]{2}\-[0-9]{2})', fileName).group(0)

def valid_date_check(date):
  if date==None or date==" " or date=="":
      return False
  else:
    date_cpy=date
    date=date.split("/")
    try:
      month=int(date[0])
      day= int(date[1])
      year=int(date[2])
      if year>=2006 and year<=2020:
        try:
          refined_date=datetime.datetime(year, month, day)
          return True
        except:
          return False
      else:
        return False
    except:
      return False

# 3. Module for time related columns

Similarly, lets check for the time as well. Here we must have time between
the standard 24 hours.

In [ ]:
#Deleting invalid time
def valid_time_check(time):
  if time==None or time==" " or time=="":
    return False
  else :
    cpy_time=time
    time=time.split(":")
    try:
      hour=int(time[0])
      mins=int(time[1])
      secs= int(time[2])
      # if hours is 24 then change it to 0 hours
      if hour == 24 and mins== 0 and secs == 0:
        hour=0
      try:
        newTime= datetime.time(hour,mins,secs)
        return True
      except :
        return False
    except:
      return False

#4. Module for Age Group and Race columns
The module works for only those columns whose column names are passed

In [ ]:
def refine_age_group_race(df_spark, victim_age_group=None, suspect_age_group=None, suspect_race=None, victim_race=None):
  #params: dataframe, col names for the respective age, gender cols
  if victim_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_age_group])
  if suspect_age_group:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_age_group])
  if suspect_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_race])
  if victim_race:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_race])
  return df_spark

# 5. Module for Gender, Race Columns for suspects and victims

The module works for only those columns whose column names are passed

In [ ]:
def refine_sex_gender_impute(df_spark, suspect_age=None, suspect_gender=None, victim_age=None, victim_gender=None):
  #params: dataframe, col names for the respective age, gender cols
  if suspect_age:
    df_spark=df_spark.na.fill("U",subset=[suspect_age])
  if victim_age:
    df_spark=df_spark.na.fill("U",subset=[victim_age])
  if suspect_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[suspect_gender])
  if victim_gender:
    df_spark = df_spark.na.fill("UNKNOWN",subset=[victim_gender])
  return df_spark

# 6.a: Module for Precinct, Jurisdiction Code:
  dropping the null values

  The module works for only those columns whose column names are passed along with the df

In [ ]:
def valid_precinct_check(precinct):
  if precinct==None or precinct==" " or precinct=="":
    return False
  else :
    return True

def valid_jur_check(jur):
  if jur==None or jur==" " or jur=="":
    return False
  else :
    return True

# 6.b Module for Reverse Geocoding the boroughs using latitudes and longitudes.

1. First we will remove the rows where latitude, longitude and boroughs are null. (around 450 tuples removed)
2. Then, where the boroughs are empty, take the latitude and longitude value and reverse geocode it using the module "reverseGeocoder".
3. Impute the borough name retrived in the empty space.


### USING MASTER DATASET
In the case of geocoding, geocoder gives us the zipcodes based on the latitude and longitude values. Inturn, we can use the master dataset of zipcodes inorder to retrive the borough names



NOTE: The dataset can be downloaded from : https://data.beta.nyc/en/dataset/pediacities-nyc-neighborhoods/resource/7caac650-d082-4aea-9f9b-3681d568e8a5

In [ ]:
def reverse_geo_code_boros(df_spark, Latitude, Longitude, Boro, lat_index, long_index):
  #select data where we have to impute
  df_temp_boro_clean=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Longitude].isNotNull()))
  boro_cleaner=df_temp_boro_clean.filter((df_temp_boro_clean[Boro].isNull())|(df_temp_boro_clean[Boro]=='NEW YORK'))

  # print("We have "+ str(boro_cleaner.count())+ " points to impute")
  print("___intializing Zip Code Look up ____")
  print("____ imputing the points ____")


  #use your path for master dataset here.
  df_zips=pd.read_csv(dst)
  zip_master={}
  zips=df_zips['zip']
  boro=df_zips['borough']
  for i, j in zip(zips, boro):
    zip_master[i]=j
  zip_master[10020]='Manhattan'
  zip_master[11249]='Brooklyn'

  def reverseGeoCoder(latitude, longitude):
    loc=geocoder.reverse(str(latitude)+', '+str(longitude), timeout=1000)
    zipCode=str(loc).split(",")[2][-5:]
    if not int(zipCode) in zip_master:
      boro="UNKNOWN"
    else:
      boro=zip_master[int(zipCode)]
    boro=boro.upper()
    return boro

  #creating UD function
  ud_func= udf(reverseGeoCoder, StringType())
  boro_cleaned_dataframe = boro_cleaner.withColumn(Boro, ud_func(boro_cleaner[lat_index], boro_cleaner[long_index]))

  #joining the imputed dataset to the maindataset and returning

  joiner_dataset=df_spark.filter((df_spark[Latitude].isNotNull()) & (df_spark[Boro]!='NEW YORK') & (df_spark[Longitude].isNotNull()) & (df_spark[Boro].isNotNull()))
  fin_df=joiner_dataset.union(boro_cleaned_dataframe)
  return fin_df

The size of dataset ~ 5M tuples. So, we need around 5000 data points for 95% confidence level with 1% interval. The size of data is almost 0.1% of the data. So we can get it into our df now

In [ ]:
df_spark=spark.read.option("header",True).csv(fn_dst,inferSchema=True)
df_spark=df_spark.sample(0.0005)
df_spark.count()

2587

# PROFILING TO CHECK FOR NULL VALUES IN ALL THE COLUMNS

In [ ]:
pandasDF = df_spark.toPandas()
ds=stream(pandasDF)
#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
SUMMONS_KEY,2587,0,2587,1.000000,11.337064
SUMMONS_DATE,2587,0,1936,0.748357,10.787426
OFFENSE_DESCRIPTION,2587,0,171,0.066100,5.067939
LAW_SECTION_NUMBER,2587,32,149,0.058317,4.924939
LAW_DESCRIPTION,2587,38,18,0.007062,2.244057
SUMMONS_CATEGORY_TYPE,2587,42,48,0.018861,3.837397
AGE_GROUP,2587,43,6,0.002358,1.921182
SEX,2587,46,4,0.001574,0.758864
RACE,2587,211,9,0.003788,0.517321
JURISDICTION_CODE,2587,0,3,0.001160,0.580238


## a. Select the columns that are common with the original dataset:
1. SUMMONS_DATE
2. AGE_GROUP
3. SEX
4. RACE
5. JURISDICTION_CODE
6. BORO
7. PRECINCT_OF_OCCUR
8. Latitude
9. Longitude

We can consider the primary key along with this
1. SUMMONS_KEY


In [ ]:
interested_columns_1=['SUMMONS_KEY', 'SUMMONS_DATE', 'AGE_GROUP', 'SEX', 'RACE', 'JURISDICTION_CODE', 'BORO', 'PRECINCT_OF_OCCUR', 'Latitude', 'Longitude']
df_spark=get_area_of_interest(df_spark, interested_columns_1)

## b. Lets pass the dataset through the preprocessing pipeline

In [ ]:
df_temp=df_spark.rdd

1. Date validation

In [ ]:
df_temp_=df_temp.map(lambda x:(x, valid_date_check(str(x[1])))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

2. Age, Race, Gender Validation

In [ ]:
# #as this code requires the pyspark dataframe(Not the rdd)
df_temp=df_temp.toDF(schema=df_spark.schema)
df_temp=refine_age_group_race(df_temp, None, 'AGE_GROUP', 'RACE', None)
df_temp=refine_sex_gender_impute(df_temp, None, "SEX", None, None)

3. Geocoding

In [ ]:
#geospacial attributes imputation
# df_temp=df_temp.toDF(schema=df_spark.schema)
df_spk=reverse_geo_code_boros(df_temp, 'Latitude', 'Longitude', 'BORO', -2, -1)

___intializing Zip Code Look up ____
____ imputing the points ____


4. Jurisdiction Code, Precinct check

In [ ]:
df_temp=df_spk.rdd
df_temp_=df_temp.map(lambda x:(x, valid_precinct_check(x[7]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

df_temp_=df_temp.map(lambda x:(x, valid_jur_check(x[5]))).filter(lambda x: x[1]==True)
df_temp=df_temp_.map(lambda x: x[0])

df_spark=df_temp.toDF(schema=df_spark.schema)

Lets profile the data now.

In [ ]:
pandasDF = df_spk.toPandas()
ds=stream(pandasDF)

#Creating profile of our dataset
profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
SUMMONS_KEY,2586,0,2586,1.000000,11.336507
SUMMONS_DATE,2586,0,1935,0.748260,10.786656
AGE_GROUP,2586,0,6,0.002320,1.970337
SEX,2586,0,5,0.001933,0.872249
RACE,2586,0,9,0.003480,0.483582
JURISDICTION_CODE,2586,0,3,0.001160,0.580394
BORO,2586,0,6,0.002320,2.132198
PRECINCT_OF_OCCUR,2586,0,77,0.029776,6.022809
Latitude,2586,0,2000,0.773395,10.638214
Longitude,2586,0,2001,0.773782,10.638988


### NOTE: as per the "reverse_geo_code_boros" module, BORO Values are categorized into 'UNKOWNN' values where Latitudes and Longitudes are NULL. This means, we are NOT removing the rows where BORO is NULL.